In [ ]:
import argilla as rg
import pandas as pd
from tqdm.auto import tqdm
from cleaner import Cleaner, Corretor

In [ ]:
rg.init(
        api_url='http://procyon.tce.pi.gov.br:6902',
        api_key='owner.apikey',
        workspace='victor_silva'
    )

In [ ]:
dados = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/objetosParaRotular.csv")
dados

In [ ]:
dados.rename(columns={"objeto": "text", "TIPO_OBJETO": "label"}, inplace=True)
dados

In [ ]:
dados = dados[dados['label'] >= 1]
dados = dados[dados['label'] <= 4]
dados

In [ ]:
def executaLimpeza(dataframe: pd.DataFrame, column: str, cased, accents) -> pd.DataFrame:
    """
    Executa a limpeza do DataFrame usando as classes Cleaner e Corretor.
    """
    sentencesDrop = []
    for indice in tqdm(dataframe.index, desc="Executando a Limpeza", colour='green'):
        texto = dataframe[column][indice]
        if isinstance(texto, str):
            texto = Cleaner().clear(texto)
            texto = Corretor(cased, accents).corrige_termos(texto)
            dataframe.at[indice, column] = texto
        else:
            sentencesDrop.append(indice)
    dataframe = dataframe.drop(sentencesDrop)
    return dataframe

In [ ]:
dados = executaLimpeza(dados, "text", cased=True, accents=True)

In [ ]:
dados.dropna(subset=['text'], inplace=True)
dados.drop_duplicates(subset=['text'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados['tamanho'] = dados['text'].apply(lambda x: len(x.split()))
dados['tamanho'].describe()

In [ ]:
dados = dados[dados['tamanho'] >= 10]
dados = dados[dados['tamanho'] <= 512]
dados

In [ ]:
classes = ['Obras e serviços de engenharia', 'Material permanente', 'Material de consumo', 'Serviços']

In [ ]:
mapping_labels = {1: classes[0], 2: classes[1], 3: classes[2], 4: classes[3]}
dados['label'] = dados['label'].map(mapping_labels)

In [ ]:
records = []
for index in tqdm(dados.index, desc="Rotulando os dados", colour='yellow'):
    prediction = dados['label'][index]
    records.append(
        rg.TextClassificationRecord(
            id = index,
            text=dados['text'][index],
            status='Default',
            annotation = prediction,
            multi_label=True
        )
    )

In [ ]:
settings = rg.TextClassificationSettings(classes)
rg.configure_dataset("objetostiposcased", settings, workspace="victor_silva")
rg.log(records, "objetostiposcased", workspace="victor_silva")